<a href="https://colab.research.google.com/github/Hrishikesh-Harsh/Text_Classification_IR/blob/main/Text_Classifier_IR.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Use TF-IDF as prescribed
### Treat Abstract, Key and Title differently
### Order: $Key > Abstract ≈ Title$
### Hence, $W_k > W_a \approx W_t$
### For TF, we can use $TF = 1 + log(n_t*W_t + n_k*W_k + n_a*W_a)$
### One More Hyper-parameter to vary is Window size for (Word,Word) pairs
### Use $3$ different Window sizes for $Key, Abstract, Title$: $Win_k, Win_a, Win_t$

In [ ]:
!git clone https://github.com/yao8839836/text_gcn.git

In [ ]:
cd text_gcn

In [ ]:
#run
!pip3 install scipy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
!python3 remove_words.py 20ng

In [ ]:
!python build_graph.py 20ng

In [1]:
#run
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
#run
import csv
import math
import nltk
import re
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
from tqdm.notebook import tqdm
import scipy.sparse as sp
nltk.download('stopwords')
nltk.download('punkt')
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize 

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [ ]:
#run
import tensorflow as tf

In [3]:
#run
# Loading original .csv file

file = open('/content/drive/MyDrive/IR_Project/dataset/PubMed.csv',encoding='Latin1')
type(file)
csvreader = csv.reader(file)

In [4]:
#run
# Set of stopwords to be removed
stop_words = set(stopwords.words('english'))

In [ ]:
#run
train_ratio = 0.7
test_ratio = 0.3
N = 207

train_N = int(train_ratio*N)
test_N = int(test_ratio*N)

In [5]:
#run
# Just the first line of the .csv file (Column Names)
header = []
header = next(csvreader)

In [6]:
#run
# vocab_words would be a set of all distinct words found in the dataset/.csv file minus the stopwords
vocab_words = set()
vocab_words_list = []

In [7]:
#run
rows = []
index = 0
title = []
keyword = []
abstract = []
for r in csvreader: 
  r_sub = []
  r[2] = r[2].replace('.',' ')
  r[2] = r[2].replace(',',' ')
  r[2] = r[2].replace(';',' ')
  r[2] = r[2].replace('|',' ')
  r[2] = r[2].replace('<',' ')
  r[2] = r[2].replace('>',' ')
  r[2] = r[2].replace(':',' ')
  r[2] = r[2].replace('=',' ')
  r[2] = r[2].replace('(',' ')
  r[2] = r[2].replace(')',' ')
  r[2] = r[2].replace('[',' ')
  r[2] = r[2].replace(']',' ')
  r[2] = r[2].replace('?',' ')


  r[3] = r[3].replace('.',' ')
  r[3] = r[3].replace(',',' ')
  r[3] = r[3].replace(';',' ')
  r[3] = r[3].replace('|',' ')
  r[3] = r[3].replace('<',' ')
  r[3] = r[3].replace('>',' ')
  r[3] = r[3].replace(':',' ')
  r[3] = r[3].replace('=',' ')
  r[3] = r[3].replace('(',' ')
  r[3] = r[3].replace(')',' ')
  r[3] = r[3].replace('[',' ')
  r[3] = r[3].replace(']',' ')
  r[3] = r[3].replace('?',' ')

  r[4] = r[4].replace('.',' ')
  r[4] = r[4].replace(',',' ')
  r[4] = r[4].replace(';',' ')
  r[4] = r[4].replace('|',' ')
  r[4] = r[4].replace('<',' ')
  r[4] = r[4].replace('>',' ')
  r[4] = r[4].replace(':',' ')
  r[4] = r[4].replace('=',' ')
  r[4] = r[4].replace('(',' ')
  r[4] = r[4].replace(')',' ')
  r[4] = r[4].replace('[',' ')
  r[4] = r[4].replace(']',' ')
  r[4] = r[4].replace('?',' ')

  words_title = word_tokenize(r[2])     # Tokenize the Title of that doc (row)
  words_keyword = word_tokenize(r[3])   # Tokenize the Keywords of that doc (row)
  words_abstract = word_tokenize(r[4])  # Tokenize the Abstract of that doc (row)

  title.append([])
  keyword.append([])
  abstract.append([])

  w_t = ""
  w_k = ""
  w_a = ""

  ''' 
      - Adding all non-stop-words to the vocabulary
      - Also maintaining doc-wise collection of Keywords, Title and Abstract words 
  '''

  for w in words_title:
    if w not in stop_words and w!="'s":
        w_t = w_t + w + " "
        vocab_words.add(w)
        # vocab_words_list.append(w)
        title[index].append(w)
  
  for w in words_keyword:
    if w not in stop_words and w!="'s":
        w_k = w_k + w + " "
        vocab_words.add(w)
        # vocab_words_list.append(w)
        keyword[index].append(w)

  for w in words_abstract:
    if w not in stop_words and w!="'s":
        w_a = w_a + w + " "
        vocab_words.add(w)
        # vocab_words_list.append(w)
        abstract[index].append(w)
  
  index=index+1

  '''
    - Creating 'rows' to write back to Clean File
  '''
  for i in range(0,len(r)):
    if(i==2):
      r_sub.append(w_t)
    elif(i==3):
      r_sub.append(w_k)
    elif(i==4):
      r_sub.append(w_a) 
    else:
      r_sub.append(r[i])
    
  rows.append(r_sub)


In [ ]:
#run
train_docs = []
test_docs = []

train_docs = rows[:train_N]
test_docs = rows[train_N:]

In [8]:
#run
for wd in vocab_words:
  vocab_words_list.append(wd)

In [9]:
#run
# Globals
docs_size = len(rows)
vocab_size = len(vocab_words)
wt_k = 3    # Weight to be given to Keywords in tf score
wt_t = 1    # Weight to be given to Title in tf score
wt_a = 1    # Weight to be given to Abstract in tf score
window_size = 20 # Window size for PMI calculation

In [ ]:
'''
  To write back to Clean File
'''
file = open('/content/drive/MyDrive/IR_Project/dataset/PubMed_Clean.csv', 'w', newline='')
writer = csv.writer(file)
writer.writerow(header)

for r in rows:
  writer.writerow(r)


In [10]:
#run
'''
  Create a dict to store {word: (ID, idf)} mapping
'''
dict_vocab = {}

for i,w in enumerate(vocab_words):
  dict_vocab[w]=(i,0)

In [11]:
for i,w in enumerate(vocab_words):
  if(i>5):
    break
  print(w)
print(vocab_size)

Thinking
Categorical
arcoma
odorant
Northeast
pandemic-A
59464


In [ ]:
file = open('/content/drive/MyDrive/IR_Project/dataset/idf.csv', 'w', newline='') 
writer = csv.writer(file) 
writer.writerow(["word","idf"])
for wd in dict_vocab: 
  count = 0
  for i,doc in enumerate(rows):
    flag = 0 
    for w_t in title[i]:
      if(wd==w_t):
        count=count+1
        flag = 1
        break 

    if(flag==1):
      continue 

    for w_k in keyword[i]:
      if(wd==w_k):
        count=count+1
        flag = 1
        break

    if(flag==1):
      continue 

    for w_a in abstract[i]:
      if(wd==w_a):
        count=count+1
        flag = 1
        break

    if(flag==1):
      continue

  (id,idf) = dict_vocab[wd]
  if(count==0):
    idf = 0
  else:
    idf = math.log((docs_size/count),10) 
  dict_vocab[wd] = (id,idf)
  templine=[]
  templine.append(wd)
  templine.append(idf)
  writer.writerow(templine)
  print(id,templine)
  count = 0
writer.writerow("fine")
file.close()

In [12]:
#run
file = open('/content/drive/MyDrive/IR_Project/dataset/idf.csv',encoding='Latin1')
type(file)
csvreader = csv.reader(file)

header = []
header = next(csvreader)

In [13]:
#run
for val in dict_vocab: 
  r = next(csvreader)
  if(r[0] not in dict_vocab):
    continue
  if(r[1]=='i'):
    break
  (id,idf) = dict_vocab[r[0]]
  idf = float(r[1])
  dict_vocab[r[0]] = (id,idf)

In [14]:
for i,ele in enumerate(dict_vocab):
  if(i==5):
    break;
  
  print(ele,",",dict_vocab[ele])

Thinking , (0, 3.344048239108943)
Categorical , (1, 3.6450782347729245)
arcoma , (2, 3.9461082304369057)
odorant , (3, 3.6450782347729245)
Northeast , (4, 3.344048239108943)


In [29]:
#run
Adj_Matrix = np.zeros((docs_size+vocab_size,docs_size+vocab_size))
weights = []
row_list = []
col_list = []

In [30]:
#run
for i,doc in enumerate(rows):
  for w_k in keyword[i]:
    Adj_Matrix[i,dict_vocab[w_k][0]+docs_size] += wt_k
    Adj_Matrix[dict_vocab[w_k][0]+docs_size,i] += wt_k

  for w_t in title[i]:
    Adj_Matrix[i,dict_vocab[w_t][0]+docs_size] += wt_t
    Adj_Matrix[dict_vocab[w_t][0]+docs_size,i] += wt_t

  for w_a in abstract[i]:
    Adj_Matrix[i,dict_vocab[w_a][0]+docs_size] += wt_a
    Adj_Matrix[dict_vocab[w_a][0]+docs_size,i] += wt_a
  # # wt = 0
  # for w_k in keyword[i]:
  #   # wt += wt_k
  #   # weights.append(wt*dict_vocab[w_k][1])
  #   row_list.append(i)
  #   col_list.append(dict_vocab[w_k][0]+docs_size)
  #   x=pow(10,Adj_Matrix(i,dict_vocab[w_k][0]+docs_size)-1)
  #   Adj_Matrix[i][dict_vocab[w_k][0]+docs_size]=x+wt_k
  #   Adj_Matrix[i][dict_vocab[w_k][0]+docs_size]=1+math.log(Adj_Matrix[i][dict_vocab[w_k][0]+docs_size])
  # for w_t in title[i]:
  #   # wt += wt_t
  #   # weights.append(wt*dict_vocab[w_t][1])
  #   row_list.append(i)
  #   col_list.append(dict_vocab[w_t][0]+docs_size)
  # for w_a in abstract[i]:
  #   # wt += wt_a
  #   # weights.append(wt*dict_vocab[w_a][1])
  #   row_list.append(i)
  #   col_list.append(dict_vocab[w_a][0]+docs_size)


In [17]:
# run
for i in range(0,docs_size):
  for j in range(docs_size,Adj_Matrix.shape[0]):
    if(Adj_Matrix[i,j]!=0):
      wd = vocab_words_list[j-docs_size]
      idf = dict_vocab[wd][1]
      Adj_Matrix[i,j] = (1+math.log(Adj_Matrix[i,j],10))*idf
      Adj_Matrix[j,i]=Adj_Matrix[i,j]


4411


In [ ]:
# word co-occurence with context windows
windows = []

for row in rows:
    content=row[2]+" "+row[3]+" "+row[4]
    words = content.split()
    length = len(words)
    if length <= window_size: 
        windows.append(words)
    else:
        # print(length, length - window_size + 1)
        for j in range(length - window_size + 1):
            window = words[j: j + window_size]
            windows.append(window)
            # print(window)

print(len(windows))

#calculating p(i) , word_window_freq has the number of windows a particular word appears in across all windows.
word_window_freq = {}
k = 0
for window in windows:
    appeared = set()
    for i in range(len(window)):
        if window[i] in appeared:
            continue
        if window[i] in word_window_freq:
            word_window_freq[window[i]] += 1
        else:
            word_window_freq[window[i]] = 1
        appeared.add(window[i])
    # print("k=",k)  
    # k+=1
print(len(word_window_freq))

word_pair_count = {}
k1=0
for window in windows:
    for i in range(1, len(window)):
        for j in range(0, i):
            word_i = window[i] #ith word in window
            word_i_id = dict_vocab[word_i][0]
            word_j = window[j] #jth word in range 0-i in the same window
            word_j_id = dict_vocab[word_j][0]
            if word_i_id == word_j_id:
                continue
            word_pair_str = str(word_i_id) + ',' + str(word_j_id) #concat id and use it to count a pair or p(i,j)
            if word_pair_str in word_pair_count: #word_pair_count stores number of pairs along with number of times they appear.
                word_pair_count[word_pair_str] += 1
            else:
                word_pair_count[word_pair_str] = 1
            # two orders
            word_pair_str = str(word_j_id) + ',' + str(word_i_id)
            if word_pair_str in word_pair_count:
                word_pair_count[word_pair_str] += 1
            else:
                word_pair_count[word_pair_str] = 1
    # k1+=1
    # print("k1=",k1)    
print(len(word_pair_count))

# row = []
# col = []
# weight = []

# pmi as weights



829112
59464
13380994


In [ ]:
num_window = len(windows)

for key in word_pair_count:
    temp = key.split(',')
    i = int(temp[0])
    j = int(temp[1])
    # print("pair(i,j)=",i,j)
    count = word_pair_count[key] #p(i,j)
    word_freq_i = word_window_freq[vocab_words_list[i]] #p(i)
    word_freq_j = word_window_freq[vocab_words_list[j]] #p(j)
    pmi = math.log((1.0 * count / num_window) /(1.0 * word_freq_i * word_freq_j/(num_window * num_window)),10) #adj(i,j)
    if pmi <= 0:
        continue
    # Adj_Matrix[docs_size+i][docs_size+j]=pmi
    weights.append(pmi)
    row_list.append(docs_size+i)
    col_list.append(docs_size+j)


In [ ]:
print(len(weights))
print(len(row_list))

0
0


In [ ]:
file = open('/content/drive/MyDrive/IR_Project/dataset/pmi.csv', 'w', newline='') 
writer = csv.writer(file) 
writer.writerow(["weight","row","col"])
for i,weight_pmi in enumerate(weights):
  line=[]
  line.append(weights[i])
  line.append(row_list[i])
  line.append(col_list[i])
  writer.writerow(line)
file.close()

In [ ]:
print(len(windows))

In [32]:
#run
file = open('/content/drive/MyDrive/IR_Project/dataset/pmi.csv',encoding='Latin1')
type(file)
csvreader = csv.reader(file)

header = []
header = next(csvreader)


In [33]:
#run
weights=[]
row_list=[]
col_list=[]
count_skip=0
for r in csvreader:
    weights.append(float(r[0]))
    row_list.append(int(r[1]))
    col_list.append(int(r[2]))
file.close()
print(weights[:5])
print(row_list[:5])
print(col_list[:5])
print(len(weights))

[0.28479792351521793, 0.28479792351521793, 0.2759803469745833, 0.2759803469745833, 0.3413138889916103]
[19693, 40771, 19693, 44805, 17238]
[40771, 19693, 44805, 19693, 40771]
11410620


In [ ]:
print(len(weights))

11410620


In [34]:
#run
from scipy.sparse import csr_matrix
Adj_Mat=csr_matrix(Adj_Matrix)

In [35]:
#run
Adj_Matrix = sp.csr_matrix((weights, (row_list, col_list)), shape=(docs_size+len(vocab_words_list), docs_size+len(vocab_words_list)))

# build symmetric adjacency matrix
Adj_Matrix = Adj_Matrix + Adj_Matrix.T.multiply(Adj_Matrix.T > Adj_Matrix) - Adj_Matrix.multiply(Adj_Matrix.T > Adj_Matrix)

In [36]:
#run
Adj_Matrix=Adj_Mat+Adj_Matrix

In [37]:
#run
Adj_Matrix=csr_matrix(Adj_Matrix)

In [38]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [39]:
if torch.cuda.is_available:
  print('GPU available')
else:
  print('Please set GPU via Edit -> Notebook Settings.')


GPU available


In [40]:
def normalise(Adj_Matrix):
    """Symmetrically normalize adjacency matrix."""
    Adj_Matrix = sp.coo_matrix(Adj_Matrix)
    rowsum = np.array(Adj_Matrix.sum(1))
    D = np.power(rowsum, -0.5).flatten()
    D[np.isinf(D)] = 0.
    D_Factor = sp.diags(D)
    return Adj_Matrix.dot(D_Factor).transpose().dot(D_Factor).tocoo(), D
    
Adj_Matrix, D = normalise(Adj_Matrix + sp.eye(Adj_Matrix.shape[0]))


def spToTensor(mx):
    """Convert a scipy sparse matrix to a torch sparse tensor."""
    mx = mx.tocoo().astype(np.float32)
    indices = torch.from_numpy(
        np.vstack((mx.row, mx.col)).astype(np.int64))
    values = torch.from_numpy(mx.data)
    shape = torch.Size(mx.shape)
    return torch.sparse.FloatTensor(indices, values, shape).to(device)

Adj_Matrix = spToTensor(Adj_Matrix)

In [41]:
torch.save(Adj_Matrix, '/content/drive/MyDrive/IR_Project/dataset/Adj.pt')

tensor(indices=tensor([[    0, 11042, 11344,  ..., 61987, 65415, 68296],
                       [    0,     0,     0,  ..., 68296, 68296, 68296]]),
       values=tensor([0.0115, 0.0143, 0.0102,  ..., 0.0021, 0.0086, 0.0245]),
       device='cuda:0', size=(68297, 68297), nnz=12896703,
       layout=torch.sparse_coo)

In [ ]:
#run
weights=[]
row_list=[]
col_list=[]
Adj_Mat=[]
windows=[]
word_window_freq = {}
word_pair_count = {}